<a href="https://colab.research.google.com/github/ankritRisal/Finetuning_LLM/blob/main/Train_finetunned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Play with data



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

from torch.utils.data import Dataset, DataLoader

from google.colab import userdata
from huggingface_hub import login

# Get the token from Colab secrets
hf_token = userdata.get('HF_TOKEN')

login(token=hf_token)


In [4]:
# model_id = "meta-llama/Llama-3.2-3B-Instruct"
model_id = "meta-llama/Llama-3.2-1B-Instruct"

device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained(model_id, padding_side ="left")
tokenizer.pad_token = tokenizer.eos_token # padding tokens to make of same shape

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             dtype = torch.bfloat16,
                                             device_map = device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/146 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

# Visualization of prediction

In [49]:
# WHAT DO BATCH PROMPT SHOULD INCLUDE ? => CHAT_TEMPLATE + " " + CATEGORY_TEMPLATE
# CATEGORY TEMPLATE => TITLE AND DESCRIPTION WITH VALID CATEGORY AS ANSWER (SHOULD OPERATE IN LOOP)
#   SYSTEM PROMPT , PORT PROMPT > CATEGORY TEMPLATE

In [12]:
df = pd.read_csv("/content/drive/MyDrive/X3s4c5/FinetunningLLMmodels/book_description.csv")
df.drop(["Unnamed: 0", "Unnamed: 0.1", "Price", "Avilability", "Stars", ], axis = 1, inplace = True)

In [27]:
def build_prompt(row):
    valid_category = list(df["Category"].unique())
    SYSTEM_PROMPT =  \
      {
        "role" : "system",
        "content" : f""" You are an AI system that reads an Title and Book Description and classifies category of the book applied, you must
        choose from the following classes:
        {"\n or ". join(["Labeled Category:" + x for x in list(valid_category)])}.
        Ensure Output is from above list only"""
        }

    ASSISTANT_MESSAGES = \
      {
        "role" : "assistant",
        "content" : "Labeled Category :"
        }

    USER_MESSAGES = {
        "role": "user",
        "content": f"""Title: {row['Title']}
Description: {row['Book_Description']}"""
    }

    prompt = [
        SYSTEM_PROMPT,
        USER_MESSAGES,
        ASSISTANT_MESSAGES
    ]
    # print(prompt)
    tokenized_prompt = tokenizer.apply_chat_template(prompt, continue_final_message= True, tokenize= False)
    return tokenized_prompt

In [14]:
random_seed = 32
train_size = 0.95
df["prompt"] = df.apply(build_prompt, axis=1)
df = df.sample(frac=1, random_state = random_seed).reset_index(drop=True).reset_index()
train_len = int(train_size * len(df))
df_train= df[:train_len]
df_test = df[train_len:]

In [15]:
# def get_dataset(tokenizer, train_size, build_prompt):
#   df = pd.read_csv("/content/drive/MyDrive/X3s4c5/FinetunningLLMmodels/book_description.csv")

#   df.drop(["Unnamed: 0", "Unnamed: 0.1", "Price", "Avilability", "Stars", ], axis = 1, inplace = True)

#   df["prompt"] = df.apply(build_prompt, axis=1)
#   df = df.sample(frac=1, random_state = random_seed).reset_index(drop=True).reset_index()
#   train_len = int(train_size * len(df))
#   df_train= df[:train_len]
#   df_test = df[train_len:]
#   # print(df_test)
#   return df_train, df_test

In [16]:
def extract_labels_from_output(decode_batch, tokenizer):
  labels = []
  for d in decode_batch:
    # print("++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    # print(d)
    label = d.split("Labeled Category :")[1].strip()
    # print(label)
    labels.append(label)
  # labels = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[0].split("<|eot_id|>")[0].strip() for d in decode_batch]
  # print(labels)
  # labels  = [d.split("<|start_header_id|>assistant<|end_header_id|>\n\nLabeled Category:")[1].split("<|eot_id|>")[0].strip() for d in decode_batch]
  return labels

In [17]:
def generate_outputs(prompts, model, tokenizer):
  tokenizer.pad_token = tokenizer.eos_token
  tokenized = tokenizer(prompts, padding= True, return_tensors ="pt", add_special_tokens= False).to(device)

  # tokenized = tokenizer.apply_chat_template(prompts, padding= False, return_tensors ="pt", add_special_tokens= False).to(device)
  output_batch = model.generate(input_ids = tokenized["input_ids"], attention_mask =tokenized["attention_mask"], max_new_tokens = 20, do_sample= False, temperature = 0, top_p =1)
  # output_batch = model.generate(tokenized, max_new_tokens = 20)
  decode_batch = tokenizer.batch_decode(output_batch, skip_special_tokens= True)
  prediction = extract_labels_from_output(decode_batch, tokenizer)
  return prediction

In [18]:
def test_model(dataloader, model, tokenizer):
  comparison_df= {
      "predictions": [],
      "labels": []
  }

  for batch in dataloader:
    predictions = generate_outputs(prompts= batch["prompt"], model = model, tokenizer= tokenizer) # prompts = batch["prompt"]
    comparison_df["labels"].extend(batch["Category"])
    comparison_df["predictions"].extend(predictions)

  # comparison_df = pd.DataFrame(comparison_df)
  # accuracy = (comparison_df["labels"] == comparison_df["predictions"]).mean()
  # num_invalid_pred = (~comparison_df["predictions"].isin(valid_category)).mean()
  # print(comparison_df.head(10))
  # return {"accuracy ": accuracy,"invalid_predictions": num_invalid_pred}
  return comparison_df

In [22]:
from datasets import Dataset #defined by huggingface not pandas or tf
batch_size = 16
test_Dataset = Dataset.from_pandas(df_test)
test_dataloader = DataLoader(test_Dataset, batch_size = batch_size, shuffle= False)
metrics = test_model(test_dataloader, model, tokenizer)
# print("\n".join([f"{k} = {v}" for k, v in metrics.items()]))

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [20]:
test_Dataset

Dataset({
    features: ['index', 'Title', 'Category', 'Book_Description', 'prompt'],
    num_rows: 50
})

In [25]:
valid_category = list(df["Category"].unique())
comparison = pd.DataFrame(metrics)
accuracy = (comparison["labels"] == comparison["predictions"]).mean()
num_invalid_pred = (~comparison["predictions"].isin(valid_category)).mean()
print(comparison.head(10))
print(f"accuracy : {accuracy}, invalid_predictions: {num_invalid_pred}")

          predictions          labels
0            Thriller         Fiction
1  Historical Fiction         History
2       Autobiography         Default
3         Young Adult  Sequential Art
4             Fiction        Classics
5              Poetry      Nonfiction
6             Mystery         Default
7              Poetry  Food and Drink
8     Science Fiction      Nonfiction
9               Music         Default
accuracy : 0.16, invalid_predictions: 0.1


In [64]:
title_name = "Ouran High School Host Club, Vol. 1 (Ouran High School Host Club #1)"
df[df["Title"] == title_name]

,index,Title,Category,Book_Description,prompt
470,470,"Ouran High School Host Club, Vol. 1 (Ouran Hig...",Sequential Art,"One day, Haruhi, a scholarship student at excl...",<|begin_of_text|><|start_header_id|>system<|en...


dataloader is object but we need list

In [ ]:
# 2758,
#            264,   4068,

In [70]:
print(tokenizer.convert_ids_to_tokens(2758))
# print(tokenizer.convert_tokens_to_ids("user"))
print(tokenizer.convert_ids_to_tokens(264))
print(tokenizer.convert_ids_to_tokens(4068))


ĠAdd
Ġa
Ġcomment


In [77]:
def generate_input_output_pair(batch_prompts, batch_targets):

  full_response_text = [
      (b_prompt + " " + target + tokenizer.eos_token)
      for b_prompt, target in zip(batch_prompts, batch_targets)
      ]
  input_ids_tokenized = tokenizer(full_response_text, add_special_tokens = False, return_tensors ="pt", padding =True)["input_ids"]
  label_tokenized = tokenizer([" " + target + tokenizer.eos_token for target in batch_targets], add_special_tokens = False,
                              return_tensors ="pt", padding = "max_length", max_length = input_ids_tokenized.shape[1])["input_ids"]

  label_tokenized_fixed = torch.where(label_tokenized != tokenizer.pad_token_id, label_tokenized, -100)
  label_tokenized_fixed[:, -1] = tokenizer.eos_token_id

  input_ids_tokenized_left_shifted = input_ids_tokenized[:, :-1]
  label_tokenized_right_shifted = label_tokenized_fixed[:, 1:]

  attention_mask = input_ids_tokenized_left_shifted != tokenizer.pad_token_id

  return {
      "input_ids" : input_ids_tokenized_left_shifted,
      "attention_mask" : attention_mask,
      "labels" : label_tokenized_right_shifted
  }

In [106]:
import torch.nn as nn
from torch.optim import AdamW

def calculate_loss(logits, labels):
  loss_fn = nn.CrossEntropyLoss(reduction= 'none')
  entropyloss = loss_fn(logits.view(-1, logits.size(-1)), labels.reshape(-1))
  return entropyloss

# **LORA ADAPTOR**

In [121]:
# LORA ADAPTOR
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    task_type = "CAUSAL_LM",
    r = 4,
    lora_alpha = 16,
    lora_dropout = 0.05,
    target_modules = ['q_proj', 'v_proj']
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 425,984 || all params: 1,236,240,384 || trainable%: 0.0345


# **DEFINE TRAINING LOOP**

In [122]:
# DURING TRAINING: TOKENIZED VALUE MUST BE COMPARED WITH LABELED VALUE
# HOW TO DO IT ?
# USE NEW DF PROMPT FOR EACH ONE

In [123]:
from datasets import Dataset #defined by huggingface not pandas or tf
batch_size = 16
train_Dataset = Dataset.from_pandas(df_train)
train_dataloader = DataLoader(train_Dataset, batch_size = batch_size, shuffle= False)

In [124]:
for batch in train_dataloader :
  data = generate_input_output_pair(batch_prompts= batch['prompt'], batch_targets= batch['Category'])
  out = model(input_ids = data["input_ids"].to(device))
  loss = calculate_loss(out.logits, data["labels"].to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.65 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.38 GiB is free. Process 8484 has 13.36 GiB memory in use. Of the allocated memory 12.60 GiB is allocated by PyTorch, and 655.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
epochs = 1
optimizer = AdamW(model.parameters(), lr = 1e-3, weight_decay= 0.01)

for epoch in range(epochs):
  running_loss = 0.0

  for batch in train_dataloader :
    data = generate_input_output_pair(batch_prompts= batch['prompt'], batch_targets= batch['Category'])
    out = model(input_ids = data["input_ids"].to(device))
    loss = calculate_loss(out.logits, data["labels"].to(device))

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    running_loss += loss.item()

  avg_loss = running_loss / len(train_dataloader)

In [79]:
for batch in test_dataloader:
  data = generate_input_output_pair(batch_prompts= batch['prompt'], batch_targets= batch['Category'])
  data["input_ids"]

In [85]:
print(data["input_ids"].shape)
print(data["labels"].shape)

torch.Size([2, 803])
torch.Size([2, 803])


In [97]:
loss = calculate_loss(out.logits, data["labels"].to(device))

In [ ]:
def train_model(dataloader, model, tokenizer):
  for batch in dataloader:
    data = generate_input_output_pair(prompt= batch['prompt'], target_responses= batch['Category'])